In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure

### Load dataset

In [ ]:
year_list = [2016, 2017, 2018, 2019, 2021, 2022]

In [ ]:
df_nonpev = pd.read_excel('data/AFVReport.xlsx', sheet_name='NJAFVnonPEVRprtbyZIPDetls202106')
df_pev = pd.read_excel('data/AFVReport.xlsx', sheet_name='NJAFVPEVRprtbyZIPDetails202206')
df_nonpev_22 = pd.read_excel('data/AFVReport.xlsx', sheet_name='NJAFVnonPEVRprtbyZIPDetls202206')
df_govt = pd.read_excel('data/AFVReport.xlsx', sheet_name='NJAFVRprtGvtVehicles202206')
df_census = pd.read_excel('data/NJ Census Data.xlsx')

### Concat dataframe

In [ ]:
df_combine = pd.concat([df_nonpev, df_pev, df_nonpev_22]).reset_index(drop=True)

### Data cleaning

In [ ]:
df_combine = df_combine[(df_combine['County']!='Unknown') & 
                        (df_combine['County']!='zzUnknown') & 
                       (df_combine['County']!='Total') &
                       ~(df_combine['County'].isnull()) &
                       ~(df_combine['ModelYear'].isnull())].reset_index(drop=True)
df_combine["ModelYear"] = df_combine["ModelYear"].astype(int)

### Filter data

In [ ]:
df_filter = df_combine[['County', 'ModelYear', 'ZEVClass', 'VehClass', 'CountOfVIN']]

### Question 1: AFV growth in different counties

In [ ]:
zev_class = list(df_filter['ZEVClass'].unique())
df_filter_q1 = df_filter[df_filter['ZEVClass'].isin(zev_class)].reset_index(drop=True)

In [ ]:
def get_year_wise_info(df, zev_class, year_list):
    county_list = list(df_filter['County'].unique())
    county_dict = []
    
    for county in county_list:
        df_county = df[df['County'] == county]
        
        for year in year_list:
            df_year = df_county[df_county['ModelYear']<=year]
            
            #zev_info = []
            for zev in zev_class:
                df_zev = df_year[df_year['ZEVClass'] == zev]
                county_dict.append([county, year, zev, df_zev['CountOfVIN'].sum()])
        #county_dict[county] = zev_info
    return county_dict

def get_count_zev_info(df):
    grp = df.groupby(by=['County', 'Year'])
    c_list = []
    
    for index, grp_df in grp:
        cnt_hev = grp_df[grp_df['ZEVClass'] == 'HEV']['CountOfVIN'].sum()
        cnt_nev = grp_df[grp_df['ZEVClass'] == 'NEV']['CountOfVIN'].sum()
        cnt_phev = grp_df[grp_df['ZEVClass'] == 'PHEV']['CountOfVIN'].sum()
        cnt_bev = grp_df[grp_df['ZEVClass'] == 'BEV']['CountOfVIN'].sum()
        c_list.append([index[0], index[1], cnt_hev, cnt_nev, cnt_phev, cnt_bev])
    
    df_final = pd.DataFrame(c_list, columns=['County', 'Year', 'HEV', 'NEV', 'PHEV', 'BEV'])
    df_final = df_final.sort_values(by=['County', 'Year', 'HEV', 'NEV', 'PHEV', 'BEV'], 
                                    ascending=True).reset_index(drop=True)
    return df_final
        

def plot_stack_bar(df):
    
    grp_item = df.groupby(by=['County'])
    
    for index, grp_df in grp_item:
        grp_df = grp_df.reset_index(drop=True)
        #return grp_df
        #set seaborn plotting aesthetics
        sns.set(rc={'figure.figsize':(6,4)})
        sns.set(style='white')

        #create stacked bar chart
        grp_df.set_index('Year').plot(kind='bar', stacked=True, color=['steelblue', 'green', 'gray', 'orange'])

        #add overall title
        plt.title(index+ ' County Electric Vehicle Growth', fontsize=16)

        #add axis titles
        plt.xlabel('Year')
        plt.ylabel('EV count')

        #rotate x-axis labels
        plt.xticks(rotation=45)
    

In [ ]:
county_dict = get_year_wise_info(df_filter, zev_class, year_list)

In [ ]:
county_df = pd.DataFrame(county_dict, columns=['County', 'Year' ,'ZEVClass', 'CountOfVIN'])
county_plot_df = get_count_zev_info(county_df)

In [ ]:
plot_stack_bar(county_plot_df)

### Q2:  Link between AFV growth and county’s rank

In [ ]:
def get_county_wise_ev(df, df_census):
    df_census['County'] = df_census['County'].str.replace(' County','')
    grp = df.groupby(by=['County'])
    c_list = []
    
    for index, grp_df in grp:
        ev_count = grp_df['CountOfVIN'].sum()
        c_list.append([index, ev_count])
    
    df_final = pd.DataFrame(c_list, columns=['County', 'Total AVF'])
    df_join = df_census.merge(df_final, on =['County'], how='left')
    return df_join

In [ ]:
df_soci = get_county_wise_ev(county_df, df_census)

In [ ]:
#df_soci['Auction_Rank'] = df_soci.groupby('County')['Total AVF'].rank()
df_soci['AVF_Rank'] = df_soci['Total AVF'].rank().astype(int)